## MNIST Network Point of Comparison


In CS345 you will notice an emphasis on always trying "more obvious" approaches.  After experimenting with fully connected networks applied to the MNIST dataset it is worthwhile to see how a simpler k nearest neighbors algorithm performs.  

However, what you will quicklly discover if you simply take what you've already done with k nearest neighbor support in sklearn is that training then testing can take up to a half hour to run.  That alone is teaching you something very important - a brute force implementation of k nearest neighbors will **NOT** scale up well to large datasets.

What you might next wonder is who is interested in making fast k nearest neighbor algorithms, and the answer is many people, including at Facebook. Which leads us to the [faiss](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) library.  Considering the scope of CS 345 we will not review exactly how 'faiss' works - if you choose reading up on it might prove very worthwhile. 

We can simply take advantage of the hard work done by others and use 'faiss'.  If you are using Conda there here you will find the  [Conda instructions](https://anaconda.org/conda-forge/faiss).

The rest of this notebook uses a simple 'FaissKNeighbors' object class written by 
Jakub Adamczyk and available through [here](https://gist.github.com/j-adamczyk/74ee808ffd53cd8545a49f185a908584#file-knn_with_faiss-py). Also, credit where credit is due, I came across this approach and suggestion in the article [Make kNN 300 times faster than Scikit-learn’s in 20 lines!](https://towardsdatascience.com/make-knn-300-times-faster-than-scikit-learns-in-20-lines-5e29d74e76bb)

Ross Beveridge

Last Update 11/16/2021

In [1]:
import numpy as np
import faiss

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import keras

class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X):
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return predictions

Using TensorFlow backend.


In [2]:
# This will download an 11.5 MB file to ~/.keras/datasets/
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [3]:

X_train_flat = X_train.reshape(-1, 784)
X_test_flat = X_test.reshape(-1, 784)

neigh = FaissKNeighbors(k=3)
neigh.fit(X_train_flat, y_train)

y_pred = neigh.predict(X_test_flat)

np.sum(y_test == y_pred)/y_test.shape[0]



0.9705